In [144]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import scipy as sp
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold

Reading data:

In [145]:
def read_data(folder, name1, name2, name3):
    loc = "%s/%s.txt" %(folder, name1)
    train_set = pd.read_csv(loc, sep="\t")
    print "Training set has %s rows and %s coumns" %(train_set.shape[0], train_set.shape[1])
    loc = "%s/%s.txt" %(folder, name2)
    test_set = pd.read_csv(loc, sep="\t")
    print "Testing set has %s rows and %s coumns" %(test_set.shape[0], test_set.shape[1])
    loc = "%s/%s.txt" %(folder, name3)
    val_set = pd.read_csv(loc, sep="\t")
    print "Validation set has %s rows and %s coumns" %(val_set.shape[0], val_set.shape[1])
    return train_set, test_set, val_set



In [146]:
train_set, test_set, val_set = read_data("data5", "train_data", "test_data", "valid_data")

Training set has 428 rows and 25 coumns
Testing set has 142 rows and 25 coumns
Validation set has 142 rows and 25 coumns


In [147]:
train_set_np = train_set.values
test_set_np = test_set.values
val_set_np = val_set.values

In [148]:
def np_cleaner(train, test, val):
    train = train[~np.isnan(train).any(axis=1)]
    print "Training set has %s rows and %s coumns" %(train.shape[0], train.shape[1])
    test = test[~np.isnan(test).any(axis=1)]
    print "Testing set has %s rows and %s coumns" %(test.shape[0], test.shape[1])
    val = val[~np.isnan(val).any(axis=1)]
    print "Validation set has %s rows and %s coumns" %(val.shape[0], val.shape[1])
    return train, test, val

In [149]:
train_set_nb, test_set_nb, val_set_nb = np_cleaner(train_set_np, test_set_np, val_set_np)

Training set has 428 rows and 25 coumns
Testing set has 142 rows and 25 coumns
Validation set has 142 rows and 25 coumns


In [150]:
train_all, test_all, val_all = np_cleaner(train_set_np, test_set_np, val_set_np)

Training set has 428 rows and 25 coumns
Testing set has 142 rows and 25 coumns
Validation set has 142 rows and 25 coumns


In [151]:
train_set.columns

Index([u'GP', u'GS', u'MIN', u'FGM', u'FGA', u'FG%', u'3PM', u'3PA', u'3P%',
       u'FTM', u'FTA', u'FT%', u'OFF', u'DEF', u'TRB', u'AST', u'STL', u'BLK',
       u'PF', u'TOV', u'PTS', u'YR', u'POS', u'W', u'H'],
      dtype='object')

In [152]:
columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 

# Labels

In [153]:
def np_labeliser(train, test, val, col):
    train = train[:,col]
    test = test[:,col]
    val = val[:,col]
    return train, test, val

In [154]:
 train_labels, test_labels, val_labels = np_labeliser(train_set_nb, test_set_nb, val_set_nb, 22)

# Feature Selection

In [155]:
def np_featuriser(train, test, val,feature_list):
    
    train = np.delete(train,feature_list,1)
    test = np.delete(test,feature_list,1)
    val = np.delete(val,feature_list,1)
    
    return train, test, val

In [156]:
feature_list = [22]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

# Variance Threshold

In [158]:
def vt_fsel(train):
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    sel.fit_transform(train)
    vt_list = sel.get_support()
    l_vt = []
    j = 0
    for i in vt_list:
        if i == False:
            l_vt.append(j)
            print "%s. featue name: %s" %(j, columns.keys()[columns.values().index(j)])
        j = j+1
    return l_vt    

In [159]:
list_vt = vt_fsel(train_all)

5. featue name: FG%
8. featue name: 3P%
11. featue name: FT%


In [160]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=10)
X = train_features_nb
y = train_labels
#print X[:1]
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
#print scores
scores /= scores.max()
print scores

#for i in scores:
#scores

print "Most valuable features: "
remove = []
for i in range(0,25):
    remove.append(i)
    
j = 0
for i in scores:
    
    if i>0.20:
        print j, i, columns.keys()[columns.values().index(j)]
        remove.remove(j)
    if j == 21:
        j = j+2
    else:
        j = j+1
#for i in range(0,25)
print remove

[ 0.01470311  0.0074998   0.04611609  0.04593729  0.08357463  0.02805854
  0.30318339  0.34438477  0.31151492  0.02153433  0.00746457  0.12981515
  0.16454504  0.05304241  0.08644591  0.33475251  0.16157611  0.15386772
  0.03772989  0.05008168  0.05688609  0.00259443  0.75265577  1.        ]
Most valuable features: 
6 0.30318339431 3PM
7 0.344384768543 3PA
8 0.311514923218 3P%
15 0.334752506224 AST
23 0.752655766218 W
24 1.0 H
[0, 1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22]


In [161]:
feature_list = [0, 1, 2, 3, 4, 6, 10, 11, 14, 19, 20, 21]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

In [162]:
#feature_list = [columns["GP"],columns["GS"],columns["MIN"],columns["PTS"],columns["FG%"],
#     columns["3P%"],columns["FT%"],columns["YR"],columns["POS"],columns['3PM'],columns['FTM'],columns['FGM']]
#train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

# Naive Bayes

In [163]:
clfNB = GaussianNB()

In [164]:
clfNB.fit(train_features_nb, train_labels)

GaussianNB()

In [165]:
predNB = clfNB.predict(val_features_nb)

# Accuracy Naive Bayes

In [166]:
accNB = metrics.accuracy_score(val_labels,predNB)
accNB
# first try, with all features - 66%
# second, GP, GS, PTS, FG%, 3P%, FT%, YR, POS, 3PM, FTM, FGM removed - 73%
# third - Univariate feature selection "Select K best" - 47% only two features GP i GS
# fourth - Univariate feature selection "SelectPercentile, f_classif" 74,6%
######THIS WAS WITH TRAIN FEATURES!!!!

1.0

# Confusion Matrix Naive Bayes

In [167]:
confm_nb = metrics.confusion_matrix(val_labels, predNB)
confm_nb
# labels 1 - 2 - 3 - 4 - 5 

array([[20,  0,  0,  0,  0],
       [ 0, 26,  0,  0,  0],
       [ 0,  0, 22,  0,  0],
       [ 0,  0,  0, 35,  0],
       [ 0,  0,  0,  0, 39]])

# SVM

In [168]:
clfSVM= SVC()

In [169]:
clfSVM.set_params(kernel = 'linear')

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [170]:
clfSVM.fit(train_features_nb, train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [171]:
predSVM = clfSVM.predict(val_features_nb)

# Accuracy SVM

In [172]:
accSVM = metrics.accuracy_score(val_labels,predSVM)
accSVM
# first try - 78% same features as nb
# second try - 82,4% kernel='linear'

1.0

# Confusion Matrix SVM

In [173]:
confm_svm = metrics.confusion_matrix(val_labels, predSVM)
confm_svm

array([[20,  0,  0,  0,  0],
       [ 0, 26,  0,  0,  0],
       [ 0,  0, 22,  0,  0],
       [ 0,  0,  0, 35,  0],
       [ 0,  0,  0,  0, 39]])

# Logistic Regression

In [174]:
lr_feature_list = [22]
train_features_lr, test_features_lr, val_features_lr = np_featuriser(train_set_nb, test_set_nb, val_set_nb,lr_feature_list)

lr_feature_list = [0, 1, 2, 3, 18, 19, 20, 21, 22]
train_features_lr, test_features_lr, val_features_lr = np_featuriser(train_set_nb, test_set_nb, val_set_nb,lr_feature_list)


#columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
#              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 

In [175]:
#for i in remove:
#    print i
#    print columns.keys()[columns.values().index(i)]

In [176]:
logreg = linear_model.LogisticRegression(C=1e5)

In [177]:
logreg.fit(train_features_lr, train_labels)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0)

In [178]:
predLR = logreg.predict(val_features_lr)

# Accuracy Logistic Regression

In [179]:
accLR = metrics.accuracy_score(predLR, val_labels)
accLR
# first try 74,6%
# 88,7% sa fsel

0.66901408450704225

# Confusion Matrix Logistic Regression

In [180]:
confm_lr = metrics.confusion_matrix(val_labels, predLR)
confm_lr

array([[16,  3,  0,  1,  0],
       [ 5, 14,  6,  1,  0],
       [ 0,  4, 14,  4,  0],
       [ 1,  1,  4, 18, 11],
       [ 1,  0,  2,  3, 33]])

In [181]:
list_none = [22]
list_domain = [columns["GP"],columns["GS"],columns["MIN"],columns["FG%"],
      columns["3P%"],columns["FT%"],columns["PTS"],columns["YR"],columns["POS"],columns['3PM'],columns['FTM'],columns['FGM']]
list_uni = [0, 1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22]

In [182]:
def print_metrics(name, accuracy, conf_matrix):
    print "Feature selection: %s\n" %name
    print "Accuracy score: %s\n" %accuracy
    print "Confusion matrix:"
    print "\n%s" %conf_matrix
    print"\n"
    
def clf(clf, tr, tr_labels, val, val_labels):
        clf.fit(tr, tr_labels)
        
        pred = clf.predict(val)
        
        acc = metrics.accuracy_score(val_labels,pred)
        conf = metrics.confusion_matrix(val_labels, pred)
        
        return acc, conf

In [183]:
def clf_all(CLF, l_none, l_domain, l_uni, train_all, test_all, val_all):
    tr_none, ts_none, val_none = np_featuriser(train_all, test_all, val_all, l_none)
    print tr_none.shape
    tr_domain, ts_domain, val_domain = np_featuriser(train_all, test_all, val_all, l_domain)
    
    tr_uni, ts_uni, val_uni = np_featuriser(train_all, test_all, val_all, l_uni)
   
    
    #clfnb = GaussianNB()
    print "Naive Bayes\n"
    
    acc_none, conf_none = clf(CLF, tr_none, train_labels, val_none, val_labels)
    print_metrics("None", acc_none, conf_none)
    
    acc_domain, conf_domain = clf(CLF, tr_domain, train_labels, val_domain, val_labels)
    print_metrics("Domain knowledge", acc_domain, conf_domain)

    acc_uni, conf_uni = clf(CLF, tr_uni, train_labels, val_uni, val_labels)
    print_metrics("Univariate", acc_uni, conf_uni)

In [200]:
def clf_all(CLF, l_none, l_domain, l_uni, l_vt, train_all, test_all, val_all):
    tr_none, ts_none, val_none = np_featuriser(train_all, test_all, val_all, l_none)
    print tr_none.shape
    tr_domain, ts_domain, val_domain = np_featuriser(train_all, test_all, val_all, l_domain)
    
    tr_uni, ts_uni, val_uni = np_featuriser(train_all, test_all, val_all, l_uni)
    
    tr_vt, ts_vt, val_vt = np_featuriser(train_all, test_all, val_all, l_vt)
    
    #clfnb = GaussianNB()
    #print "Naive Bayes\n"
    
    acc_none, conf_none = clf(CLF, tr_none, train_labels, ts_none, test_labels)
    print_metrics("None", acc_none, conf_none)
    
    acc_domain, conf_domain = clf(CLF, tr_domain, train_labels, ts_domain, test_labels)
    print_metrics("Domain knowledge", acc_domain, conf_domain)

    acc_uni, conf_uni = clf(CLF, tr_uni, train_labels, ts_uni, test_labels)
    print_metrics("Univariate", acc_uni, conf_uni)
    
    acc_vt, conf_vt = clf(CLF, tr_vt, train_labels, ts_vt, test_labels)
    print_metrics("Variance Threshold", acc_vt, conf_vt)

In [201]:
clfnb = GaussianNB()
clf_all(clfnb, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

(428L, 24L)
Feature selection: None

Accuracy score: 0.661971830986

Confusion matrix:

[[13  0  0  0  0]
 [11 15  3  0  2]
 [ 0  3 14  0  1]
 [ 0  0  1 16 19]
 [ 0  0  0  8 36]]


Feature selection: Domain knowledge

Accuracy score: 0.725352112676

Confusion matrix:

[[13  0  0  0  0]
 [10 19  2  0  0]
 [ 0  2 16  0  0]
 [ 0  0  2 15 19]
 [ 0  0  0  4 40]]


Feature selection: Univariate

Accuracy score: 0.683098591549

Confusion matrix:

[[13  0  0  0  0]
 [ 6 21  3  0  1]
 [ 0  3 13  1  1]
 [ 0  0  3  7 26]
 [ 0  0  0  1 43]]


Feature selection: Variance Threshold

Accuracy score: 1.0

Confusion matrix:

[[13  0  0  0  0]
 [ 0 31  0  0  0]
 [ 0  0 18  0  0]
 [ 0  0  0 36  0]
 [ 0  0  0  0 44]]




In [187]:
svm = SVC()
svm.set_params(kernel='linear')
clf_all(svm, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

(428L, 24L)
Feature selection: None

Accuracy score: 0.711267605634

Confusion matrix:

[[18  2  0  0  0]
 [ 8 15  3  0  0]
 [ 0  3 15  4  0]
 [ 0  0  7 21  7]
 [ 0  0  0  7 32]]


Feature selection: Domain knowledge

Accuracy score: 0.753521126761

Confusion matrix:

[[18  2  0  0  0]
 [ 2 19  5  0  0]
 [ 0  4 13  5  0]
 [ 0  0  4 22  9]
 [ 0  0  0  4 35]]


Feature selection: Univariate

Accuracy score: 0.725352112676

Confusion matrix:

[[17  3  0  0  0]
 [ 2 16  8  0  0]
 [ 0  4 12  6  0]
 [ 0  0  3 23  9]
 [ 0  0  0  4 35]]


Feature selection: Variance Threshold

Accuracy score: 0.992957746479

Confusion matrix:

[[20  0  0  0  0]
 [ 0 26  0  0  0]
 [ 0  0 22  0  0]
 [ 0  0  1 34  0]
 [ 0  0  0  0 39]]




In [188]:
logreg = linear_model.LogisticRegression(C=1e5)
clf_all(logreg, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

(428L, 24L)
Feature selection: None

Accuracy score: 0.570422535211

Confusion matrix:

[[14  4  2  0  0]
 [ 6 12  6  2  0]
 [ 2  3 13  4  0]
 [ 0  1  5 11 18]
 [ 0  0  2  6 31]]


Feature selection: Domain knowledge

Accuracy score: 0.669014084507

Confusion matrix:

[[16  4  0  0  0]
 [ 7 14  2  3  0]
 [ 1  5 10  6  0]
 [ 0  2  1 19 13]
 [ 0  0  0  3 36]]


Feature selection: Univariate

Accuracy score: 0.598591549296

Confusion matrix:

[[15  2  1  2  0]
 [ 6  7  6  7  0]
 [ 0  4 12  6  0]
 [ 0  1  3 15 16]
 [ 0  0  3  0 36]]


Feature selection: Variance Threshold

Accuracy score: 0.802816901408

Confusion matrix:

[[18  2  0  0  0]
 [ 0 18  6  2  0]
 [ 0  2 13  7  0]
 [ 0  0  9 26  0]
 [ 0  0  0  0 39]]


